Obtain

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.style.use('seaborn')
df = pd.read_csv('kc_house_data.csv')
df.head()

Scrub

In [ ]:
df.info() # check datatypes

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(subset=['view'])

Explore

In [ ]:
sns.heatmap(df.corr());

In [ ]:
(abs(df.corr()) > 0.70).sum()

In [ ]:
df.hist(figsize=(10,10));

In [ ]:
df = df[df.bedrooms <= 6]
df = df[df.bathrooms <= 5]
df = df[df.condition >= 2]
df = df[df.grade > 3]
df.price = df.price[df.price <= 700000]
df.sqft_above = df.sqft_above[df.sqft_above <= 3000]
df.sqft_living = df.sqft_living[df.sqft_living < 4000]
df.sqft_living15 = df.sqft_living15[df.sqft_living15 <= 3600]
df.sqft_lot = df.sqft_lot[df.sqft_lot <= 25000]
df.sqft_lot15 = df.sqft_lot15[df.sqft_lot15 <= 20000]
df.hist(figsize=(10,10));

Model

• Overall: Develop a model that can predict a target from given inputs
• use simple ols to assess accuracy of model thus far
•

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
column_names = df.describe().columns.drop(['price'])
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for i, column_name in enumerate(column_names):
    print ("King County: price ~ " + column_name)
    print ("------------------------------")

    f = 'price~' + column_name
    model = smf.ols(formula=f, data=df).fit()
    X_new = pd.DataFrame({column_name: [df[column_name].min(), df[column_name].max()]});
    preds = model.predict(X_new)
    results.append([column_name, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[i+1])

In [ ]:
results_df = pd.DataFrame(results)
results_df.columns = results_df.iloc[0,:]
results_df.drop(labels=results_df.index[0], axis=0, inplace=True)

r_coef = pd.DataFrame((results_df['r_squared'])**0.5)
r_coef.rename(columns={"r_squared":"r_coef"}, inplace=True)
results_df = pd.concat([results_df, r_coef], axis=1)

p_bool = pd.DataFrame(results_df['p-value'] < 0.05)
p_bool.rename(columns={"p-value":"p_bool"}, inplace=True)
results_df = pd.concat([results_df, p_bool], axis=1)

r_coef_bool = pd.DataFrame(results_df['r_coef'] > 0.50)
r_coef_bool.rename(columns={"r_coef":"r_coef_bool"}, inplace=True)
results_df = pd.concat([results_df, r_coef_bool], axis=1)

results_df

Interpret